In [1]:
from typing import Dict
import os

from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from monai.networks.nets import DenseNet121
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torchvision

/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:93: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. 

In [2]:
pl.seed_everything(7)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64
NUM_WORKERS = int(os.cpu_count() / 2)

Global seed set to 7


In [3]:
train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

cifar10_dm = CIFAR10DataModule(
    data_dir=PATH_DATASETS,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    val_transforms=test_transforms,
)

In [4]:
class DenseNetModel(pl.LightningModule):
    def __init__(
        self,
        learning_rate: float = 0.05,
        in_channels: int = 3,
        num_classes: int=10
    ):
        super().__init__()

        # store hyperparameters
        self.save_hyperparameters()
        
        self.loss_function = CrossEntropyLoss()
        
        # make the model
        self.model = DenseNet121(spatial_dims=2, in_channels=in_channels, out_channels=num_classes)
        
    def forward(self, x) -> torch.Tensor:
        return self.model(x)
    
    def _compute_loss_from_batch(self, batch: Dict[str, torch.Tensor], batch_idx: int) -> float:
        inputs = batch[0]
        labels = batch[1]
        
        outputs = self.model(inputs)
        return self.loss_function(outputs, labels)
    
    def training_step(
        self, batch: Dict[str, torch.Tensor], batch_idx: int
    ) -> Dict[str, float]:
        # compute the loss
        loss = self._compute_loss_from_batch(batch=batch, batch_idx=batch_idx)
        
        # perform logging
        self.log("training_loss", loss, batch_size=len(batch[0]))
        
        return {"loss": loss}

    def validation_step(self, batch: Dict[str, torch.Tensor], batch_idx: int) -> float:
        return self._compute_loss_from_batch(batch=batch, batch_idx=batch_idx)
    
    def test_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs).argmax(dim=-1)
        acc = (labels == preds).float().mean()
        # By default logs it per epoch (weighted average over batches), and returns it afterwards
        self.log("test_acc", acc)
    
    def configure_optimizers(self) -> Adam:
        return Adam(self.model.parameters(), lr=self.hparams.learning_rate)

In [5]:
model = DenseNetModel(learning_rate=0.05)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    logger=CSVLogger(save_dir="logs/"),
    callbacks=[LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)],
)

trainer.fit(model, cifar10_dm)
trainer.test(model, datamodule=cifar10_dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


100%|████████████████████████████████████████████████████████████████████████████████| 170498071/170498071 [02:29<00:00, 1139727.65it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


Missing logger folder: logs/lightning_logs

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | model         | DenseNet121      | 7.0 M 
---------------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.856    Total estimated model params size (MB)


Epoch 0:  79%|█████████████████████████████████████████████████████▉              | 620/782 [16:09<04:13,  1.56s/it, loss=1.77, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 1:  79%|█████████████████████████████████████████████████████▉              | 620/782 [19:08<05:00,  1.85s/it, loss=1.57, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 2:  79%|█████████████████████████████████████████████████████▉              | 620/782 [15:52<04:08,  1.54s/it, loss=1.41, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 3:  79%|█████████████████████████████████████████████████████▉              | 620/782 [15:14<03:58,  1.47s/it, loss=1.32, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 4:  79%|█████████████████████████████████████████████████████▉              | 620/782 [15:56<04:09,  1.54s/it, loss=1.15, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 5:  79%|█████████████████████████████████████████████████████▉              | 620/782 [18:53<04:56,  1.83s/it, loss=1.16, v_num=0]
Validation: 0it [00:00,

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|███████████████████████████████████████████████████████████████████| 782/782 [14:39<00:00,  1.12s/it, loss=0.57, v_num=0]
Files already downloaded and verified
Files already downloaded and verified
Testing DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████| 157/157 [00:42<00:00,  3.67it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7989000082015991     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.7989000082015991}]